In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

There are some factors that affects Death Evernt. This dataset contains person information like age, sex, blood pressure, smoke etc. We have to predict their DEATH EVENT. This dataset is VERY small so its better to not treat so serious this data.

# Importing libraries

In [ ]:
import warnings
import seaborn as sns
from colorama import Fore, Back, Style
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from mlxtend.plotting import plot_confusion_matrix
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from statsmodels.formula.api import ols
import plotly.graph_objs as gobj

init_notebook_mode(connected=True)
warnings.filterwarnings('ignore')
import plotly.figure_factory as ff

%matplotlib inline

import xgboost
import lightgbm
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

# Data Exploration

In [ ]:
heart_data = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
heart_data.head()

In [ ]:
heart_data['DEATH_EVENT'].value_counts()

* Sex - Gender of patient value_counts1, Female - 0
* Age - Age of patient
* Diabetes - 0 = No, 1 = Yes
* Anaemia - 0 = No, 1 = Yes
* High_Blood_Pressure - 0 = No, 1 = Yes
* Smoking - 0 = No, 1 = Yes
* DEATH_EVENT - 0 = No, 1 = Yes

# Does Age and Sex are an indication of Death Event?

In [ ]:
#age distribution
hist_data = [heart_data['age'].values]
group_labels = ['age']

fig = ff.create_distplot(hist_data, group_labels)
fig.update_layout(title_text='Age Distribution Plot')

fig.show()

* Age wise 40 to 80 the spread is High
* less that 40 age and higher than 80 age people are very low

In [ ]:
fig = px.box(heart_data, x='sex', y='age', points='all')
fig.update_layout(
title_text='Gender wise Age spread - Male = 1, Female = 0')
fig.show()

In [ ]:
male = heart_data[heart_data['sex'] == 1] 
female = heart_data[heart_data['sex'] == 0] 

male_survi = male[heart_data['DEATH_EVENT'] == 0]
male_not = male[heart_data['DEATH_EVENT'] == 1]
female_survi = female[heart_data['DEATH_EVENT'] == 0]
female_not = female[heart_data['DEATH_EVENT'] == 1]

labels = ['Male - Survived', 'Male - Not Survived', 'Female - Survived', 'Female - Not Survived']
values = [len(male[heart_data['DEATH_EVENT'] == 0]), len(male[heart_data['DEATH_EVENT'] == 1]),
          len(female[heart_data['DEATH_EVENT'] == 0]),len(female[heart_data['DEATH_EVENT'] == 1]),]
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(
title_text = 'Analysis on Survival - Gender')
fig.show()

# Gender Factor Analysis

In [ ]:
surv = heart_data[heart_data['DEATH_EVENT'] == 0]['age']
not_surv = heart_data[heart_data['DEATH_EVENT'] == 1]['age']
hist_data = [surv, not_surv]
group_labels = ['Survived', 'Not Survived']
fig = ff.create_distplot(hist_data, group_labels, bin_size=0.5)
fig.update_layout(
title_text='Analysis in Age on Survival Status')
fig.show()

* Survival is higher from 40 to 70 years old
* The spread of Not survival is going through all ages.

In [ ]:
fig = px.violin(heart_data, y='age', x='sex', color='DEATH_EVENT', box=True,points='all',hover_data=heart_data.columns)
fig.update_layout(title_text='Analysis in Age and Gender on Survival Status')
fig.show()

Age Report:
* Survical spread is high in age flow of 40 to 70
* The Survival is high for both male between 50 to 60 and female's age between 60 to 70 respectively

In [ ]:
fig = px.violin(heart_data, y="age", x="smoking", color="DEATH_EVENT", box=True, points="all", hover_data=heart_data.columns)
fig.update_layout(title_text="Analysis in Age and Smoking on Survival Status")
fig.show()

* The Survival is high for not smoking person 55 to 65, while for smoking person it is between 50 to 60
* Death evernt for smoking person is high than not smoking person

In [ ]:
fig = px.violin(heart_data, y="age", x="diabetes", color="DEATH_EVENT", box=True, points="all", hover_data=heart_data.columns)
fig.update_layout(title_text="Analysis in Age and Diabetes on Survival Status")
fig.show()

**Hist Plots**

In [ ]:
fig = px.histogram(heart_data, x='creatinine_phosphokinase', color='DEATH_EVENT', marginal='violin',
                  hover_data=heart_data.columns)
fig.show()

In [ ]:
fig = px.histogram(heart_data, x='ejection_fraction', color='DEATH_EVENT', marginal='violin',
                  hover_data=heart_data.columns)
fig.show()

In [ ]:
fig = px.histogram(heart_data, x='platelets', color='DEATH_EVENT', marginal='violin',
                  hover_data=heart_data.columns)
fig.show()

In [ ]:
fig = px.histogram(heart_data, x="serum_creatinine", color="DEATH_EVENT", marginal="violin", hover_data=heart_data.columns)
fig.show()

**Distribution plots Servived and Not-Survived**

In [ ]:
surv = heart_data[heart_data['DEATH_EVENT']==0]['serum_sodium']
not_surv = heart_data[heart_data['DEATH_EVENT']==1]['serum_sodium']
hist_data= [surv, not_surv]
group_labels = ['Survived', 'Not Survived']
fig = ff.create_distplot(hist_data, group_labels, bin_size=0.5)
fig.update_layout(
title_text='Analysis in Serum Sodium on Survival Status')
fig.show()

In [ ]:
surv = heart_data[heart_data['DEATH_EVENT']==0]['serum_creatinine']
not_surv = heart_data[heart_data['DEATH_EVENT']==1]['serum_creatinine']
hist_data= [surv, not_surv]
group_labels = ['Survived', 'Not Survived']
fig = ff.create_distplot(hist_data, group_labels, bin_size=0.5)
fig.update_layout(
title_text='Analysis in Serum Creatinine on Survival Status')
fig.show()

In [ ]:
surv = heart_data[heart_data['DEATH_EVENT']==0]['ejection_fraction']
not_surv = heart_data[heart_data['DEATH_EVENT']==1]['ejection_fraction']
hist_data= [surv, not_surv]
group_labels = ['Survived', 'Not Survived']
fig = ff.create_distplot(hist_data, group_labels, bin_size=0.5)
fig.update_layout(
title_text='Analysis in Ejection Fraction on Survival Status')
fig.show()

**Heatmap**

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
sns.heatmap(heart_data.corr(), vmin=-1, cmap='coolwarm', annot=True);

# Data Modeling

**Train Test Split**

In [ ]:
Features = ['time', 'ejection_fraction','serum_creatinine']
x = heart_data[Features]
y = heart_data['DEATH_EVENT']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=2)

In [ ]:
accuracy_list = []

**Logistic Regression**

In [ ]:
#logistic regression

log_reg = LogisticRegression()
log_reg.fit(x_train,y_train)
log_reg_pred = log_reg.predict(x_test)
log_reg_acc = accuracy_score(y_test, log_reg_pred)
accuracy_list.append(log_reg_acc * 100)

In [ ]:
print('Accuracy of Logistic Regression is : ', '{:.2f}%'.format(100 * log_reg_acc))

In [ ]:
cm = confusion_matrix(y_test, log_reg_pred)
plt.figure()
plot_confusion_matrix(cm, figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.title('Logistic Regression Model - Confusion Matrix')
plt.xticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.yticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.show()

**Support Vector**

In [ ]:
#svc
sv_clf = SVC()
sv_clf.fit(x_train,y_train)
sv_clf_pred = sv_clf.predict(x_test)
sv_clf_acc = accuracy_score(y_test, sv_clf_pred)
accuracy_list.append(100*sv_clf_acc)

In [ ]:
print('Accuracy of Support Vector is : ', '{:.2f}%'.format(100 * sv_clf_acc))

In [ ]:
cm = confusion_matrix(y_test, sv_clf_pred)
plt.figure()
plot_confusion_matrix(cm, figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.title('SVC Model - Confusion Matrix')
plt.xticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.yticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.show()

**K-Neighbours Classifier**

In [ ]:
# K Neighbours Classifier
kn_clf = KNeighborsClassifier(n_neighbors=6)
kn_clf.fit(x_train,y_train)
kn_pred = kn_clf.predict(x_test)
kn_acc = accuracy_score(y_test,kn_pred)
accuracy_list.append(kn_acc * 100)

In [ ]:
print('Accuracy of KNeighbours is : ', '{:.2f}%'.format(100 * kn_acc))

In [ ]:
cm = confusion_matrix(y_test, kn_pred)
plt.figure()
plot_confusion_matrix(cm, figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.title('KN  Model - Confusion Matrix')
plt.xticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.yticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.show()

**Decision Tree Classifier**

In [ ]:
#Decision Tree Classifier

dt_clf = DecisionTreeClassifier(max_leaf_nodes=3, random_state=0, criterion='entropy')
dt_clf.fit(x_train,y_train)
dt_pred = dt_clf.predict(x_test)
dt_acc = accuracy_score(y_test,dt_pred)
accuracy_list.append(dt_acc * 100)

In [ ]:
print('Accuracy of DecisionTreeClassifier is : ', '{:.2f}%'.format(100 * dt_acc))

In [ ]:
cm = confusion_matrix(y_test, dt_pred)
plt.figure()
plot_confusion_matrix(cm, figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.title('Decision Tree  Model - Confusion Matrix')
plt.xticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.yticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.show()

**Random Forest Classifier**

In [ ]:
#RandomForest Classifier

r_clf = RandomForestClassifier(max_features=0.5, max_depth=15, random_state=1)
r_clf.fit(x_train,y_train)
r_pred = r_clf.predict(x_test)
r_acc = accuracy_score(y_test,r_pred)
accuracy_list.append(r_acc * 100)

In [ ]:
print('Accuracy of RandomForestClassifier is : ', '{:.2f}%'.format(100 * r_acc))

In [ ]:
cm = confusion_matrix(y_test, r_pred)
plt.figure()
plot_confusion_matrix(cm, figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.title('Random Forest Model - Confusion Matrix')
plt.xticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.yticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.show()

**Gradient Boosting Classifier**

In [ ]:
# GradientBoostingClassifier

gradientboost_clf = GradientBoostingClassifier(max_depth=2, random_state=1)
gradientboost_clf.fit(x_train,y_train)
gradientboost_pred = gradientboost_clf.predict(x_test)
gradientboost_acc = accuracy_score(y_test, gradientboost_pred)
accuracy_list.append(100*gradientboost_acc)

In [ ]:
print('Accuracy of GradientBoostingClassifier is : ', '{:.2f}%'.format(100 * gradientboost_acc))

In [ ]:
cm = confusion_matrix(y_test, gradientboost_pred)
plt.figure()
plot_confusion_matrix(cm, figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.title('GradientBoostingClassifier Model - Confusion Matrix')
plt.xticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.yticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.show()

**XGBoost Classifier**

In [ ]:
#xgboost classifier
xgb_clf = xgboost.XGBRFClassifier(max_depth=3, random_state=1)
xgb_clf.fit(x_train,y_train)
xgb_pred = xgb_clf.predict(x_test)
xgb_acc = accuracy_score(y_test, xgb_pred)
accuracy_list.append(100*xgb_acc)

In [ ]:
print('Accuracy of XGBRFClassifier is : ', '{:.2f}%'.format(100 * xgb_acc))

In [ ]:
cm = confusion_matrix(y_test, xgb_pred)
plt.figure()
plot_confusion_matrix(cm, figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.title('XGBRFClassifier Model - Confusion Matrix')
plt.xticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.yticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.show()

**LGBMClassifier**

In [ ]:
#  lightgbm 

lgb_clf = lightgbm.LGBMClassifier(max_depth=2, random_state=4)
lgb_clf.fit(x_train,y_train)
lgb_pred = lgb_clf.predict(x_test)
lgb_acc = accuracy_score(y_test, lgb_pred)
accuracy_list.append(100*lgb_acc)

In [ ]:
print('Accuracy of LGBMClassifier is : ', '{:.2f}%'.format(100 * lgb_acc))

In [ ]:
cm = confusion_matrix(y_test, lgb_pred)
plt.figure()
plot_confusion_matrix(cm, figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.title('LGBMClassifier Model - Confusion Matrix')
plt.xticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.yticks(range(2), ['Heart Not Failed', 'Heart Fail'], fontsize=16)
plt.show()

**CatBoostClassifier**

In [ ]:
# CatBoostClassifier

cat_clf = CatBoostClassifier()
cat_clf.fit(x_train,y_train)
cat_pred = cat_clf.predict(x_test)
cat_acc = accuracy_score(y_test, cat_pred)
accuracy_list.append(100*cat_acc)

In [ ]:
print("Accuracy of CatBoostClassifier is : ","{:.2f}%".format(100* cat_acc))

In [ ]:
cm = confusion_matrix(y_test, cat_pred)
plt.figure()
plot_confusion_matrix(cm, figsize=(12,8), hide_ticks=True, cmap=plt.cm.Blues)
plt.title("Cat Boost Classifier Model - Confusion Matrix")
plt.xticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.yticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.show()


In [ ]:
model_list = ['Logistic Regression', 'SVC','KNearestNeighbours', 'DecisionTree', 'RandomForest',
              'GradientBooster', 'XGBRF','LGBM', 'CatBoostClassifier']

In [ ]:
plt.rcParams['figure.figsize'] = 20, 8
sns.set_style('darkgrid')
ax = sns.barplot(x=model_list, y=accuracy_list, palette = 'husl', saturation=2.0)
plt.xlabel('Classifier Models', fontsize=20)
plt.ylabel('% of Accuracy', fontsize=20)
plt.title('Accuracy of different Classifier Models', fontsize=20)
plt.xticks(fontsize = 12, horizontalalignment = 'center', rotation = 8)
plt.yticks(fontsize = 12)
for i in ax.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy()
    ax.annotate(f'{round(height, 2)}%', ( x + width/2, y + height* 1.02), ha='center', fontsize='x-large')
    plt.show()